# Lab 1: Introduction to Machine Learning with scikit-learn

scikit-learn is an open source Python machine learning module. Using scikit-learn makes it easy to implement a variety of machine learning tasks including regression, classification, clustering, dimensionality reduction, model selection and data pre-processing. The full documentation for scikit-learn is located at https://scikit-learn.org/stable/. It contains numerous examples and detailed descriptions for each function (and its associated parameters). This notebook will give you a brief introduction on how to implement the basic machine learning pipeline.

In [ ]:
# if you are missing any of the packages, uncomment the line(s) below to install
# %pip install sklearn
# %pip install pandas
# %pip install numpy

### Loading & Previewing Data

The scikit-learn library contains a few small datasets that we can experiment with. Descriptions of the datasets are included here: https://scikit-learn.org/stable/datasets/toy_dataset.html. We will use two of them in this tutorial:

1) The Boston house prices dataset (for regression tasks) and

2) The breast cancer wisconsin (diagnostic) dataset (for classification tasks)

First we will load the datasets using helper functions and will take a look at the contents:

In [1]:
from sklearn.datasets import load_boston
import pandas as pd

boston = load_boston()
print(boston.DESCR)
(boston_X, boston_y) = load_boston(return_X_y=True)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [2]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
print(cancer.DESCR)
(cancer_X, cancer_y) = load_breast_cancer(return_X_y=True)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

### Splitting Data

Before interacting with our datasets any further, we must split the data into training and testing data. This way we can use the training data to explore the data and fit our models and then use the testing data to provide an unbiased measure of the performance of our models. This can be done using the train_test_split function in scikit-learn.

In [3]:
from sklearn.model_selection import train_test_split

# split the data with a 75%-25% training-test split
# set the random state for reproducible results

boston_X_train, boston_X_test, boston_y_train, boston_y_test = train_test_split(
    boston_X, boston_y, test_size=0.25, random_state=671)
# checks
print(boston_X_train.shape)
print(boston_X_test.shape)
print(boston_y_train.shape)
print(boston_y_test.shape)

(379, 13)
(127, 13)
(379,)
(127,)


In [4]:
# YOUR CODE: split the breast cancer data with a train-test split of 70%-30% & random_state of 671 into:

cancer_X_train, cancer_X_test, cancer_y_train, cancer_y_test = train_test_split(
    cancer_X, cancer_y, test_size=0.3, random_state=671)

# checks
print(cancer_X_train.shape)
print(cancer_X_test.shape)
print(cancer_y_train.shape)
print(cancer_y_test.shape)

(398, 30)
(171, 30)
(398,)
(171,)


### Preprocessing the Data

For almost all datasets, we will need to preprocess the data before we can fit any models. This may involve imputing missing data, scaling our features, applying one-hot encoding, etc.

#### 1. Imputing Missing Data

Missing data is a very common problem across all datasets. One simple strategy for addressing it is to impute missing values using a chosen strategy such as the "mean", "median" or "most_frequent". First, let's check for missing data in our datasets.

In [6]:
boston_X_train_df = pd.DataFrame(boston_X_train, columns=boston.feature_names)
# check for which columns have missing values
columns = boston_X_train_df.columns[boston_X_train_df.isnull().any()]
print(len(columns))

0


In [7]:
cancer_X_train_df = pd.DataFrame(cancer_X_train, columns=cancer.feature_names)
# check for which columns have missing values
columns = cancer_X_train_df.columns[cancer_X_train_df.isnull().any()]
print(len(columns))

0


As these are "toy" datasets, they do not have any missing values but let us still practice how we would impute missing values anyways.

In [11]:
from sklearn.impute import SimpleImputer
import numpy as np

# impute using the 'mean' for the feature
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# use fit_transform on training data
boston_X_train_imp = imp.fit_transform(boston_X_train)
# use transform on testing data
boston_X_test_imp = imp.transform(boston_X_test)

In [12]:
# apply the SimpleImputer with a 'median' strategy to the cancer data
# As a result, you should have two variables:
# (1) cancer_X_train_imp and
# (2) cancer_X_test_imp
from sklearn.impute import SimpleImputer
import numpy as np


# sometimes, missing_values ~= np.nan, may 0,999,-1....
imp = SimpleImputer(missing_values=np.nan, strategy='median')

# we fit and impute the trainX (fit_transform) and use same imputer to impute testX (tranform), just dont use info from testsata; also this is the reason we split first
cancer_X_train_imp = imp.fit_transform(cancer_X_train)

# 把traindata fit出来的median值impute到testdata，（当testdata不存在
cancer_X_test_imp = imp.transform(cancer_X_test)

#### 2.  One-Hot Encoding

Depending on the nature of our data and the ML models we want to implement, we may need to one-hot encode the categorical variables in our data as some models do not support categorical inputs. This means we will transform the feature into a set of dummy variables.

Although we do not have any categorical variables in our cancer dataset, we do have a variable, 'RAD', in our boston dataset that we want to one-hot encode as it represents an index of accessibility to radial highways. We can do so using a OneHotEncoder and ColumnTransfer, which allows us to only apply the OneHotEncoder to only the 'RAD' column.

First, let's implement the OneHotEncoder with the default parameters.

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

rad_idx = list(boston.feature_names).index(
    'RAD')  # find the index of the 'RAD' column = 8

# passthrough specifies to keep other columns as they are
enc_boston = ColumnTransformer(
    [("RAD", OneHotEncoder(), [rad_idx])], remainder="passthrough")

boston_X_train_enc = enc_boston.fit_transform(
    boston_X_train_imp)  # use fit_transform on training data
boston_X_test_enc = enc_boston.transform(
    boston_X_test_imp)  # use transform on testing data

print(enc_boston.transformers_)

print(boston_X_train_enc.shape)
print(boston_X_test_enc.shape)

[('RAD', OneHotEncoder(), [8]), ('remainder', 'passthrough', [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12])]
(379, 21)
(127, 21)


We can see that after applying the OneHotEncoder to the 'RAD' column, we have increased the number of columns in our dataset from 13 to 21 as it added one column for each unique value of 'RAD'. However, if we use this, we will run into the issue of multicollinearity since each dummy variable can be represented as a linear combination of the other dummy variables. To solve this issue, we want to create n-1 dummy variables. This can be done using the 'drop' parameter of the OneHotEncoder function, as shown below, resulting in one less column.

In [18]:
# drop 1 to solve issue of multicollinearity

enc_boston = ColumnTransformer(
    [("RAD", OneHotEncoder(drop='first'), [rad_idx])], remainder="passthrough")
boston_X_train_enc = enc_boston.fit_transform(boston_X_train_imp)
boston_X_test_enc = enc_boston.transform(boston_X_test_imp)
print(enc_boston.transformers_)

print(boston_X_train_enc.shape)
print(boston_X_test_enc.shape)

[('RAD', OneHotEncoder(drop='first'), [8]), ('remainder', 'passthrough', [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12])]
(379, 20)
(127, 20)


#### 3. Feature Scaling

For some machine learning models, we must first scale the features so that they are all on a shared scale. This supports faster model convergence and removes any bias toward features with higher magnitudes (i.e., giving more importance to features on a scale of cm vs. inches).


Two of the most common techniques for feature normalization are 1) StandardScaler and 2) MinMaxScaler. StandardScaler works by adjusting the mean of each feature to zero with a standard deviation of 1. MinMaxScaler works by scaling all values to between 0 and 1. There are no hard rules for when to use one over the other but some factors to consider are the problem we intend to solve, assumptions regarding the distribution of the data (including the presence of outliers) and the ML models we plan on implementing. It can also be a good option to try out both and see which results in better performance. Either way, we must fit the scaler on our training data and then transform our testing data using it to avoid any data leakage.

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
boston_X_train_scal = scaler.fit_transform(
    boston_X_train_enc)  # use fit_transform on training data
boston_X_test_scal = scaler.transform(
    boston_X_test_enc)  # use transform on testing data

In [20]:
from sklearn.preprocessing import MinMaxScaler

# apply MinMaxScaler to the cancer dataset ('cancer_X_train_imp' and 'cancer_X_test_imp')
# name the resulting variables 'cancer_X_train_scal' and 'cancer_X_test_scal'

scaler = MinMaxScaler()
cancer_X_train_scal = scaler.fit_transform(
    cancer_X_train_imp)  # use fit_transform on training data
cancer_X_test_scal = scaler.transform(
    cancer_X_test_imp)  # use transform on testing data

### Feature Selection

When we have large datasets with high dimensionality, feature selection can help us reduce the dimensionality (e.g., the number of input variables or features) by removing irrelevant or redundant features. This can help with reducing the computational costs associated with training models and can also improve the performance of our models in some cases. There are many feature selection techniques so we will only experiment with a few here.

In [22]:
# Method 1: Mutual Information (regression)
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

# select the top 10 features
selector = SelectKBest(mutual_info_regression, k=10)
boston_X_train_mi = selector.fit_transform(boston_X_train_scal, boston_y_train)
print("Training data shape (after applying Mutual Info):", boston_X_train_mi.shape)
boston_X_test_mi = selector.transform(boston_X_test_scal)
print("Testing data shape (after applying Mutual Info):", boston_X_test_mi.shape)

# Method 2: Principal component analysis (PCA)

# can specify either the percent of explained variance or number of features
pca = PCA(n_components=0.90)  # explaine 90% variance
boston_X_train_pca = pca.fit_transform(boston_X_train_scal, boston_y_train)
print("Training shape (after applying PCA):", boston_X_train_pca.shape)
boston_X_test_pca = pca.transform(boston_X_test_scal)
print("Testing shape (after applying PCA):", boston_X_test_pca.shape)

Training data shape (after applying Mutual Info): (379, 10)
Testing data shape (after applying Mutual Info): (127, 10)
Training shape (after applying PCA): (379, 12)
Testing shape (after applying PCA): (127, 12)


In [23]:
# apply Mutual Information (classification) to the breast cancer data
# select the top 15 features and name the resulting variables 'cancer_X_train_mi' & 'cancer_X_test_mi'

from sklearn.feature_selection import mutual_info_classif

# Method 1: Mutual Information (regression)
# select the top 15 features
selector = SelectKBest(mutual_info_classif, k=15)

cancer_X_train_mi = selector.fit_transform(cancer_X_train_scal, cancer_y_train)
print("Training data shape (after applying Mutual Info):", cancer_X_train_mi.shape)
cancer_X_test_mi = selector.transform(cancer_X_test_scal)
print("Testing data shape (after applying Mutual Info):", cancer_X_test_mi.shape)

# # Method 2: Principal component analysis (PCA)

# # can specify either the percent of explained variance or number of features
# pca = PCA(n_components=0.90)
# cancer_X_train_pca = pca.fit_transform(cancer_X_train_scal, cancer_y_train)
# print("Training shape (after applying PCA):", cancer_X_train_pca.shape)
# cancer_X_test_pca = pca.transform(cancer_X_test_scal)
# print("Testing shape (after applying PCA):", cancer_X_test_pca.shape)

Training data shape (after applying Mutual Info): (398, 15)
Testing data shape (after applying Mutual Info): (171, 15)


### Model Fitting, Cross-Validation & Evaluation

Now we can finally fit some models! scikit-learn allows you to easily implement a variety of models. We will work with a few of them here.

Let's start by fitting two models - (1) Linear Regression and (2) K-Nearest Neighbors Regression - to our boston housing dataset. We will fit the models to each of the three feature sets - (1) all of the features (no feature selection), (2) the feature subset selected using mutual information and (3) the feature subset selected using PCA so that we can compare performance both between the models and the different feature subsets. We will use cross-validation (3 folds in this example) to get a more reliable estimate of the performance of our models without having to touch our test dataset yet. The default performance metric will be $R^2$, or the proportion of explained variance.

In [24]:
# Cross-Validation use the data all from train dataset! not touch the test dataset

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

lr = LinearRegression()

lr_scores_df = pd.DataFrame(
    columns=["R^2 on Fold 1", "R^2 on Fold 2", "R^2 on Fold 3", "Mean R^2"])
print("Linear Regression:")

lr_scal_scores = list(cross_val_score(
    lr, boston_X_train_scal, boston_y_train, cv=3))
lr_scal_scores.append(np.mean(lr_scal_scores))
lr_scores_df.loc["No Feature Selection"] = lr_scal_scores

lr_mi_scores = list(cross_val_score(
    lr, boston_X_train_mi, boston_y_train, cv=3))
lr_mi_scores.append(np.mean(lr_mi_scores))
lr_scores_df.loc["MI Features"] = lr_mi_scores

lr_pca_scores = list(cross_val_score(
    lr, boston_X_train_pca, boston_y_train, cv=3))
lr_pca_scores.append(np.mean(lr_pca_scores))
lr_scores_df.loc["PCA Features"] = lr_pca_scores

print(lr_scores_df.head())

knn = KNeighborsRegressor()

knn_scores_df = pd.DataFrame(
    columns=["R^2 on Fold 1", "R^2 on Fold 2", "R^2 on Fold 3", "Mean R^2"])
print("\nK-Nearest Neighbors Regression:")

knn_scal_scores = list(cross_val_score(
    knn, boston_X_train_scal, boston_y_train, cv=3))
knn_scal_scores.append(np.mean(knn_scal_scores))
knn_scores_df.loc["No Feature Selection"] = knn_scal_scores

knn_mi_scores = list(cross_val_score(
    knn, boston_X_train_mi, boston_y_train, cv=3))
knn_mi_scores.append(np.mean(knn_mi_scores))
knn_scores_df.loc["MI Features"] = knn_mi_scores

knn_pca_scores = list(cross_val_score(
    knn, boston_X_train_pca, boston_y_train, cv=3))
knn_pca_scores.append(np.mean(knn_pca_scores))
knn_scores_df.loc["PCA Features"] = knn_pca_scores

print(knn_scores_df.head())

# to selct both the model and way of feature selection:
# so we pick KNN - MI Features in this case， cause it has the highest R2

Linear Regression:
                      R^2 on Fold 1  R^2 on Fold 2  R^2 on Fold 3  Mean R^2
No Feature Selection       0.779718       0.626554       0.673468  0.693247
MI Features                0.750709       0.670249       0.651748  0.690902
PCA Features               0.805717       0.546609       0.622107  0.658144

K-Nearest Neighbors Regression:
                      R^2 on Fold 1  R^2 on Fold 2  R^2 on Fold 3  Mean R^2
No Feature Selection       0.760128       0.543937       0.616316  0.640127
MI Features                0.845605       0.792886       0.722900  0.787130
PCA Features               0.769108       0.509425       0.582599  0.620377


Looking at the results, we see that linear regression performs similarily across the feature subsets but for KNN, we see that the model performs significantly better on the feature subset chosen using mutual information. Let's use GridSearchCV below to see if we can further improve its performance by adjusting the value of n_neighbors (the number of neighbors used).

In [28]:
# adjust k (hyperperemetrs

from sklearn.model_selection import GridSearchCV

knn = KNeighborsRegressor()

# search over n_neighbors
param_grid = [{'n_neighbors': [1, 2, 3, 4, 5, 6, 7]}]

cv_knn = GridSearchCV(knn, param_grid, cv=2)
cv_knn.fit(boston_X_train_mi, boston_y_train)
print("Best Params:", cv_knn.best_params_)
print("Best R^2 Score:", cv_knn.best_score_)
print(cv_knn.cv_results_)

Best Params: {'n_neighbors': 3}
Best R^2 Score: 0.8166716460160099
{'mean_fit_time': array([0.00050712, 0.        , 0.00049531, 0.0009985 , 0.00099754,
       0.        , 0.0009973 ]), 'std_fit_time': array([5.48362732e-06, 0.00000000e+00, 4.95314598e-04, 4.76837158e-07,
       2.38418579e-07, 0.00000000e+00, 0.00000000e+00]), 'mean_score_time': array([0.00049925, 0.00099719, 0.00124538, 0.00149477, 0.00099719,
       0.00149548, 0.00099742]), 'std_score_time': array([4.99248505e-04, 3.57627869e-07, 2.48074532e-04, 4.97698784e-04,
       1.19209290e-07, 4.98652458e-04, 1.19209290e-07]), 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 6, 7],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 2}, {'n_neighbors': 3}, {'n_neighbors': 4}, {'n_neighbors': 5}, {'n_neighbors': 6}, {'n_neighbors': 7}], 'split0_test_score': array([0.82304336, 0.80160726, 0.82015593, 0.8086826

In [39]:
# after selection, fitting and adjusting on train, now use the adjusted final model
# to train and test on our dataset

In [40]:
# for regression tasks, evaulate use MSE, MAE

Using GridSearchCV, we see that the the value of n_neighbors resulting in the best performance was 3. Let's use this as our final model and apply it to our test set. We will evaluate the test set using **Mean Squared Error (MSE) and Mean Absolute Error (MAE), two common measures of model performance for regression tasks**. The formula for MSE is:

MSE = $\frac{1}{n}\sum_{i=1}^n(y_i - \hat{y}_i )^2$ 

where $n$ is the number of data points, $y_i$ is the actual observation and $\hat{y}_i$ is our prediction

Similarly, the formula for MAE:

MAE = $\frac{1}{n}\sum_{i=1}^n|(y_i - \hat{y}_i )|$ 

where $n$ is the number of data points, $y_i$ is the actual observation and $\hat{y}_i$ is our prediction

In [33]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(boston_X_train_mi, boston_y_train)
preds = knn.predict(boston_X_test_mi)

print("Training R^2:", knn.score(boston_X_train_mi, boston_y_train))
print("Testing R^2:", knn.score(boston_X_test_mi, boston_y_test))
print("MSE:", mean_squared_error(boston_y_test, preds))
print("MAE:", mean_absolute_error(boston_y_test, preds))

Training R^2: 0.9391797935603915
Testing R^2: 0.8423629972031058
MSE: 12.876482939632547
MAE: 2.750656167979003


Nice! Our testing score increased to 0.84! Now, let's move onto the breast cancer data, a classification task...

In [37]:
# cross validation for cancer data (classification

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf_scores_df = pd.DataFrame(
    columns=["Fold 1 Accuracy", "Fold 2 Accuracy", "Fold 3 Accuracy", "Mean Accuracy"])
print("Random Forest Classifier:")

rf_scal_scores = list(cross_val_score(
    rf, cancer_X_train_scal, cancer_y_train, cv=3))
rf_scal_scores.append(np.mean(rf_scal_scores))
rf_scores_df.loc["All Features"] = rf_scal_scores

rf_mi_scores = list(cross_val_score(
    rf, cancer_X_train_mi, cancer_y_train, cv=3))
rf_mi_scores.append(np.mean(rf_mi_scores))
rf_scores_df.loc["MI Features"] = rf_mi_scores

print(rf_scores_df.head())


# find another classifier and try!!!!!!!!!!!
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn_scores_df = pd.DataFrame(
    columns=["Fold 1 Accuracy", "Fold 2 Accuracy", "Fold 3 Accuracy", "Mean Accuracy"])
print("\nK-Nearest Neighbors Classifier:")

knn_scal_scores = list(cross_val_score(
    knn, cancer_X_train_scal, cancer_y_train, cv=3))
knn_scal_scores.append(np.mean(knn_scal_scores))
knn_scores_df.loc["All Features"] = knn_scal_scores

knn_mi_scores = list(cross_val_score(
    knn, cancer_X_train_mi, cancer_y_train, cv=3))
knn_mi_scores.append(np.mean(knn_mi_scores))
knn_scores_df.loc["MI Features"] = knn_mi_scores


print(knn_scores_df.head())

# this accuracy is R2 score

Random Forest Classifier:
              Fold 1 Accuracy  Fold 2 Accuracy  Fold 3 Accuracy  Mean Accuracy
All Features         0.977444         0.954887         0.962121       0.964817
MI Features          0.962406         0.939850         0.954545       0.952267

K-Nearest Neighbors Classifier:
              Fold 1 Accuracy  Fold 2 Accuracy  Fold 3 Accuracy  Mean Accuracy
All Features         0.962406         0.969925         0.969697       0.967343
MI Features          0.932331         0.932331         0.954545       0.939736


This is a good example to show that feature selection will not always improve performance. This is why we must experiment with different methods and models to optimize them for the task at hand.

In [38]:
# For a lot of classification tasks: precision, recall and the f1-score

For a lot of classification tasks, we will want to focus our evaluation on precision, recall and the f1-score. The classification_report function in scikit-learn makes it easy to do so. As a review:


Precision = $\frac{TP}{TP + FP}$, or the fraction of our positive predictions that are actually positive instances

Recall = $\frac{TP}{TP + FN}$, or the fraction of positive instances that we actually predict as positive

F1-score = $2*\: \frac{precision\: *\: recall}{precision + recall}$, or the harmonic mean of precision and recall
- f1-score is basicaly: 考虑到预计错误的情况的“准确值”， 也是越大越好

In [41]:
from sklearn.metrics import classification_report

# using your chosen model, compute predictions for the test data
# use these predictions to create the classification report

knn = KNeighborsClassifier()
knn.fit(cancer_X_train_scal, cancer_y_train) # from crossva, we know all features are better than mi features
preds = knn.predict(cancer_X_test_scal)
print(classification_report(cancer_y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        68
           1       0.95      0.99      0.97       103

    accuracy                           0.96       171
   macro avg       0.97      0.96      0.96       171
weighted avg       0.97      0.96      0.96       171

